In [1]:
using Pkg

In [12]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

"C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

In [13]:
Pkg.build("CPLEX")

   Building MbedTLS → `C:\Users\Geoffroy Leconte\.julia\packages\MbedTLS\a1JFn\deps\build.log`
   Building CPLEX ──→ `C:\Users\Geoffroy Leconte\.julia\packages\CPLEX\Hxpuk\deps\build.log`


In [12]:
using CPLEX
using QPSReader
using QuadraticModels
using NLPModels

In [28]:
function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_CPlex(xpress_model)
    # show results
    sol = CPLEX.get_solution(model_cplex)
    println("soln = $(sol)")

    objv = CPLEX.get_objval(model_cplex)
    println("objv = $(objv)")
end
    

function OptimizeCPlex(qpdata)
    QM = createQuadraticModel(qpdata)
    SM = SlackModel(QM)
    
    env = CPLEX.Env()
    
    model = CPLEX.cplex_model(env; f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)
    # H = ... if quadratic problem
     # run optimization
    CPLEX.optimize!(model)
    return model
end

OptimizeCPlex (generic function with 1 method)

In [19]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [30]:
model_cplex = OptimizeCPlex(qpdata2);
display_results_CPlex(model_cplex)

soln = [80.0, 25.5, 54.5, 84.80000000000001, 18.214285714285715, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.214285714285715, 0.0, 19.30714285714286, 500.0, 475.92, 24.08, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.9428571428572, 383.9428571428572, 0.0, 0.0, 80.0, 0.0, 18.214285714285715, 0.0, -0.0, -0.0, 500.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -378.46035714285716, 299.8, 19.30714285714286]
objv = -464.7531428571428
